# DL Final Project

In [1]:
from models.transunet import TransUnet
from data_processing.data_parser import DataReader
from experiments.config import get_transunet
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X = np.load("./sampleData/trainInputsSample224.npy")
y = np.load("./sampleData/trainOutputsSample224.npy")
X_train = X.astype("float32")
y_train = y.astype("float32")
X_val = X[int(len(X)*.7):]
y_val = y[int(len(y)*.7):]
X_test = np.load("./sampleData/testInputsSample224.npy")
y_test = np.load("./sampleData/testOutputsSample224.npy")

In [4]:
## Train Model
config = get_transunet()
# config.image_size = 512
# config.decoder_channels = [512, 256, 128, 64]
# config.n_skip = 6
config.image_size = 224
config.filters = 3
print(config.image_size)
transunet = TransUnet(config)
transunet.compile()
#history = transunet.train_validate(training_dataset, validation_dataset, save_path, epochs=150)

224


In [5]:
history = transunet.model.fit(x=X_train[:40].astype("float32"), y=y_train[:40].astype("float32"), validation_data=(X_val.astype("float32"), y_val.astype("float32")), epochs=5, batch_size=20, verbose=1)

Epoch 1/5
***************** (None, 224, 224, 3)
&&&&&&&&&&&&&&&&&& (None, 224, 224, 3)
***************** (None, 224, 224, 3)
&&&&&&&&&&&&&&&&&& (None, 224, 224, 3)


In [ ]:
plt.figure()
plt.plot(history.history["loss"], label="training loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.title(f"Loss for model")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from PIL import Image as im
data = im.fromarray(y_test[0])
      
# saving the final output 
# as a PNG file
data.save('gfg_dummy_pic.png')

In [ ]:
print(y_train.shape)